# Configuracion

### Carga de librerias

In [1]:
from Metadatos import generaPathProyecto, RMSLE, logTransf, antilogTransf
from CargaDatos import retornaSets
from Preparacion import preparaDf, separacionEntrenaObjetivo
from Modelo import definirModelo
from CargaDatos import generaPath
from Salida import generarEnvio

In [2]:
import ggplot as gg

In [3]:
%matplotlib inline

In [4]:
import numpy as np

### Definicion de funciones

In [5]:
def validarModelo():
    dScores = {'best' : -modelo.best_score_,
               'train' : RMSLE(y_train, antilogTransf(modelo.predict(X_train))),
               'test' : RMSLE(y_test, antilogTransf(modelo.predict(X_test)))}
    print str('La mejor puntuacion de la crossvalidacion fue: {}\n' +
              'La puntuacion del entrenamiento fue: {}\n' +
              'La puntuacion de la prueba fue: {}').format(dScores['best'], dScores['train'], dScores['test'])

# Ejecucion de rutina

### Entrenamiento

In [6]:
path_proyecto = generaPathProyecto()
sets_df = retornaSets(path_proyecto = path_proyecto)

In [ ]:
prepDf = preparaDf()
df = prepDf.preparar(sets_df)
X_train, X_test, y_train, y_test = separacionEntrenaObjetivo(df, semilla = 1962, prop_prueba= 0.3)

In [ ]:
modelo = definirModelo()
modelo.fit(X = X_train.values, y = logTransf(y_train).values.reshape(y_train.shape[0],))

### Presentacion de resultados

In [13]:
modelo.best_params_

{'decisor__learning_rate': 0.02,
 'decisor__max_depth': 8,
 'decisor__max_features': 0.2,
 'decisor__min_samples_leaf': 7}

In [ ]:
modelo.best_score

In [12]:
validarModelo()

La mejor puntuacion de la crossvalidacion fue: 0.0827127816879
La puntuacion del entrenamiento fue: 0.0708121158123
La puntuacion de la prueba fue: 0.181625717096


### Experimentacion

### Envio

In [14]:
X_envio = prepDf.preparar(sets_df, train_o_envio= 'test')
generarEnvio(modelo, X_envio)